In [10]:
    filename = "LakeTheo.xyz"
    ascii_xyz = []
    externalfile = open(filename,'r') # create connection to file, set to read (r), file must exist
    for line in externalfile:
        if line[0] == '#' or line[0] == "D" : # skip comments and header row
            continue
        else:
            ascii_xyz.append([str(n) for n in line.strip().split()])
    externalfile.close()
    ascii_xyz_len = len(ascii_xyz) # how many rows
    print(ascii_xyz[0]) # print first row

['301493.230623291573', '3814195.86687662033', '0']


In [11]:
nrows = 214
ncols = 344
nlay = nrows*ncols # a dummy dimension
if(nrows*ncols == ascii_xyz_len):
    print('Yay! Sizes match - proceede')

Yay! Sizes match - proceede


In [12]:
ascii_RCZ = [[0 for i in range(ncols)] for j in range(nrows)] # prepopulate RC with zeros (will assign correct values below)
index=0
for irow in range(nrows):
    for jcol in range(ncols): # process as if data are column-wise (probably are)
        ascii_RCZ[irow][jcol] = float(ascii_xyz[index][2]) # put the correct elevation in place
        index = index+1 # increment the counter

# should now have the watershed in grid structure for the DTRM program, 
# not quite done yet as it needs to be written into proper format for FORTRAN program, but well along the way!

In [13]:
# Find the outlet (assume lowest cell bigger than zero)
lowElev = 99999 # start widda big number
for irow in range(nrows):
    for jcol in range(ncols):
        if(ascii_RCZ[irow][jcol] > 0): 
            if(ascii_RCZ[irow][jcol] <= lowElev):
                lowElev = ascii_RCZ[irow][jcol]
                outrow = irow
                outcol = jcol

#print(outrow,outcol,lowElev) # this is our outlet information

In [14]:
# Find the high cell(s) values
hiElev = -99999 # start widda little number
for irow in range(nrows):
    for jcol in range(ncols):
        if(ascii_RCZ[irow][jcol] > 0): 
            if(ascii_RCZ[irow][jcol] >= hiElev):
                hiElev = ascii_RCZ[irow][jcol]
                hirow = irow
                hicol = jcol

#print(hirow,hicol,hiElev) # this is our high value information of last high cell visited

In [15]:
# Now set all no-flows to hiElev
for irow in range(nrows):
    for jcol in range(ncols):
        if(ascii_RCZ[irow][jcol] <= 0): 
            ascii_RCZ[irow][jcol] = hiElev

In [16]:
# Now build the mask grid to contain the boundary location
# Mask value == 0 if no particle cell
# Mask value = 1 if particle cell
part_mask = [[0 for i in range(ncols)] for j in range(nrows)] # initialize the mask all zeros
np = 0
for irow in range(nrows):
    for jcol in range(ncols): # process as if data are column-wise (probably are)
         if (ascii_RCZ[irow][jcol] < hiElev):
                part_mask[irow][jcol]=1
                np = np + 1
                
#print(part_mask[22]) #looks like the right structure

In [164]:
#recast the list into fields of {10}.{6} so FORTRAN program does not get messed up
for irow in range(nrows):
    for jcol in range(ncols):
        ascii_RCZ[irow][jcol] = f"{ascii_RCZ[irow][jcol]:{10}.{6}}"  # builds a string with correct field width and precision (sig digits)

In [135]:
width = 10
precision = 4
value = float("12.34567")
f"result: {value:{width}.{precision}}"



DIRECTIVE 

       214       344             NROW  NCOL 



In [17]:
# Now need to write an output file in FORTRAN structure
filename = "LakeTheo.mat.dat"
externalfile = open(filename,"w") # "w" clobbers content already there, consider if "a" is better choice!
blocks = int(ncols/11)+1
#debug externalfile.write("Row Count    : " + repr(int(nrows)) + "\n")
#debug externalfile.write("Column Count : " + repr(int(ncols))+ "\n")
#debug externalfile.write("Block Count  : " + repr(int(blocks))+ "\n")
########### BEGIN FRONT MATTER #################
firstline = "DIRECTIVE \n"  # Directive(s); TERSE, VERBOSE, or DIRECTIVE(do nothing)
f1 = f"{nrows:{10}}"
f2 = f"{ncols:{10}}"
f3 = "             NROW  NCOL \n"
secondline =    f1 + f2 + f3
#debug print(firstline)
#debug print(secondline)
externalfile.write(firstline)
externalfile.write(secondline)
########### END FRONT MATTER ###################
########### BEGIN ELEVATION MATRIX #############
for irow in range(nrows): # should be a single row
#    for jcol in range(ncols): # process as if data are column-wise (probably are)
    end=-1
    for block in range(blocks):
        begin=end+1
        end = begin+10
#debug        externalfile.write("row" + repr(irow) + " block" + repr(block) + " begin" + repr(begin) + " end" + repr(end) + " ")
#        message = '  '.join(map(repr, f"{ascii_RCZ[irow][begin:end]:{10}.{6}}")) + "\n"
        message = '  '.join(map(repr, round(ascii_RCZ[irow][begin:end],3)) + "\n"
        externalfile.write(message)
########### END ELEVATION MATRIX ################
########### BEGIN SIMULATION CONTROLS ###########
cman = 1.5   # scaling term
fdeep = 0.200  # average overland flow depth (subjective 6-8 inches is upper support)
nman = 0.04  # roughness term
alife = 1.000  # survival probability for one time step as per bernoulli process
cman = 100*cman
nman = 100*nman
f1 = f"{cman:{10}.{6}}"
f2 = f"{fdeep:{10}.{6}}"
f3 = f"{nman:{10}.{6}}"
f4 = f"{alife:{10}.{6}}"
f5 = "   CMAN,FDEEP,NMAN,ALIFE \n"
externalfile.write(f1 + f2 + f3 + f4 + f5)
dx = 26.656 # cell size (meters)
dt = 1.0 # minutes
tmax = 1440.0 # minutes
d1 = 1000.0 # huh?
dxt = 26.656 # pixel size (meters)
dyt = 26.656 # pixel size (meters)
f7 = "      DX,DT,TMAX,D1,DXT,DYT \n"
f6 = f"{dyt:{10}.{6}}"
f5 = f"{dxt:{10}.{6}}"
f4 = f"{d1:{10}.{6}}" 
f3 = f"{tmax:{10}.{6}}"
f2 = f"{dt:{10}.{6}}"
f1 = f"{dx:{10}.{6}}"
externalfile.write(f1 + f2 + f3 + f4 + f5 + f6 + f7)
# compute the initial particle count 
f1 = f"{np:{10}}"
nprint=100 # print output every nprint time steps
f2 = f"{nprint:{10}}"
f3 = "            NP,PRINT_N \n "
externalfile.write(f1 + f2 + f3)
########### END SIMULATION CONTROLS #############
########### BEGIN BOUNDARY MASK #################
for irow in range(nrows): # should be a single row
#    for jcol in range(ncols): # process as if data are column-wise (probably are)
    end=-1
    for block in range(blocks):
        begin=end+1
        end = begin+10
#debug        externalfile.write("row" + repr(irow) + " block" + repr(block) + " begin" + repr(begin) + " end" + repr(end) + " ")
        message = '  '.join(map(repr, part_mask[irow][begin:end])) + "\n"
        externalfile.write(message)
########### END BOUNDARY MASK #################
########### BEGIN BACK MATTER #################
f1 = f"{outrow:{10}}"
f2 = f"{outcol:{10}}"
f3 = f"{lowElev:{10}.{6}}"
f4 = "  OUTLET LOCATION AND ELEVATION \n"
externalfile.write(f1 + f2 + f3 + f4)
########### END BACK MATTER #################
externalfile.close()
print("file for DTRM is built")

SyntaxError: invalid syntax (<ipython-input-17-331e61a5b08b>, line 29)